# Alpaca

Registrarse:


https://alpaca.markets/

Documentacion:


https://alpaca.markets/docs/

In [1]:
import requests

In [ ]:
api_key = "PK2G3TP3Y8M0VUCIJTF9"
secret_key = "IGzFJq9fX3o9S17mpa5kPbsbG7Viagsm7w57biln"

In [ ]:
def getAccount():    
    
    headers ={"APCA-API-KEY-ID" : api_key, "APCA-API-SECRET-KEY":secret_key}
    endpoint = "https://paper-api.alpaca.markets/v2/account"
    
    r = requests.get(url = endpoint, headers = headers)
    
    js = r.json()
    return js

Veamos entonces lo que nos devuelve:

In [ ]:
getAccount()

## MarketData

Vamos a ver como pedimos marketData con esta API, empecemos por la data histórica, le vamos a tener que pasar dos parámetros a la API:
   * TimeFrame
       * 1Min
       * 5Min
       * 15Min
       * 1D (day)
   * Symbolo/Simbolos

In [ ]:
import pandas as pd

def getHistory(symbol, timeframe='1D'): 
    
    headers ={"APCA-API-KEY-ID" : api_key, "APCA-API-SECRET-KEY":secret_key}
    params =  {'symbols' : symbol}
    endpoint = "https://data.alpaca.markets/v1/bars/"+timeframe
    
    r = requests.get(url = endpoint, headers =headers, params=params)
    js = r.json()
    
    df = pd.DataFrame(js[symbol])
    df.t = pd.to_datetime(df.t, unit='s')
    return df

getHistory("GGAL")

In [ ]:
import datetime as dt

#definimos una fecha como el 20 de Junio de 2020, a las 12:15hs 
fecha = dt.datetime(2020,6,20,12,15,0)

fecha

In [ ]:
fechaSRT = dt.datetime.strftime(fecha, format='%Y-%m-%dT%T-03:00')
fechaSRT

In [ ]:
import pandas as pd

def getHistory(symbol, start, end, timeframe='1D'):    
    
    start = dt.datetime.strftime(start, format='%Y-%m-%dT%T-04:00')
    end = dt.datetime.strftime(end, format='%Y-%m-%dT%T-04:00')

    headers ={"APCA-API-KEY-ID" : api_key, "APCA-API-SECRET-KEY":secret_key}
    
    params =  {'symbols' : symbol, 'start':start, 'end':end}
    
    endpoint =  "https://data.alpaca.markets/v1/bars/"+timeframe
    r = requests.get(url = endpoint, headers =headers, params=params)
    js = r.json()
    df = pd.DataFrame(js[symbol])
    df.t = pd.to_datetime(df.t, unit='s')
    return df

In [ ]:
start = dt.datetime(2012,1,1,0,0,0)
end = dt.datetime(2020,7,10,0,0,0)

getHistory('AAPL', start, end, timeframe='day')

## Bacth requests  (200 tickers x request)

### Breve repaso de funciones basicas (split, join)

In [ ]:
tickers = 'AAPL,MSFT'
tickers = tickers.split(",")
tickers

In [ ]:
lista = ['GGAL','AAPL','AMZN']
','.join(lista)

In [ ]:
import pandas as pd

def getHistory(symbol, start, end, timeframe='1D'):    
    
    start = dt.datetime.strftime(start, format='%Y-%m-%dT%T-04:00')
    end = dt.datetime.strftime(end, format='%Y-%m-%dT%T-04:00')

    headers ={"APCA-API-KEY-ID" : api_key, "APCA-API-SECRET-KEY":secret_key}
    params =  {'symbols' : symbol, 'start':start, 'end':end}
    endpoint = "https://data.alpaca.markets/v1/bars/"+timeframe
    
    r = requests.get(url = endpoint, headers =headers, params=params)
    
    js = r.json()
    
    tickers = symbol.split(",")
    dfs = {}
    for ticker in tickers:
        dfs[ticker] = pd.DataFrame(js[ticker])
        dfs[ticker].t = pd.to_datetime(dfs[ticker].t, unit='s')
    return dfs

In [ ]:
start = dt.datetime(2012,1,1,0,0,0)
end = dt.datetime(2021,1,1,0,0,0)
dfs = getHistory('AAPL,MSFT', start, end, timeframe='day')
dfs

In [ ]:
dfs['AAPL']

In [ ]:
dfs['MSFT']

## Ultimo Trade de un ticker

In [ ]:
def lastTade(symbol):    
    headers ={"APCA-API-KEY-ID" : api_key, "APCA-API-SECRET-KEY":secret_key}
    endpoint = "https://data.alpaca.markets/v1/last/stocks/"+symbol
    r = requests.get(url = endpoint, headers = headers)
    js = r.json()['last']
    js['timestamp'] = pd.to_datetime(js['timestamp'])
    return js
    
lastTade("AAPL")

## Ultimo dato del libro

In [ ]:
import pandas as pd

def lastQuote(symbol):    
    headers ={"APCA-API-KEY-ID" : api_key, "APCA-API-SECRET-KEY":secret_key}
    endpoint = "https://data.alpaca.markets/v1/last_quote/stocks/"+symbol
    r = requests.get(url = endpoint, headers = headers)
    js = r.json()['last']
    js['timestamp'] = pd.to_datetime(js['timestamp'])
    return js
    
lastQuote("AAPL")

# Ruteo de ordenes, primeros pasos de un bot

## Orden de Compra/Venta  (POST)

In [ ]:
def order(symbol, qty, side, tipo, time_in_force, limit_price ):    
    headers ={"APCA-API-KEY-ID" : api_key, "APCA-API-SECRET-KEY":secret_key}
    params = {'symbol':symbol, 
              'qty':qty, 
              'side':side, 
              'type':tipo,
              'time_in_force':time_in_force,
              'limit_price':limit_price, 
              'order_class':'simple'}
    
    r = requests.post(url = "https://paper-api.alpaca.markets/v2/orders", 
                      headers = headers, json = params)
    
    js = r.json()
    return js

In [ ]:
lastQuote("AAPL")

In [ ]:
lastTade("AAPL")

In [ ]:
px  = lastQuote("AAPL")['askprice']*0.999

orden = order(symbol='AAPL', qty=1, side='sell', tipo='limit', time_in_force='gtc',limit_price = px )
orden

¿Y si quiero mandar 50 ordenes con un par de lineas con esa lógica tapando la punta vigente?

In [ ]:
tickers = ['PAM', 'GGAL', 'BBAR', 'TGS']

for ticker in tickers:
    px  = lastQuote(ticker)['bidprice'] * 1.001
    orden = order(symbol=ticker, qty=1, side='buy', tipo='limit', time_in_force='gtc',limit_price = px )
    #print(px, order)
    #break



## Consulta de estado general

In [ ]:
def orders(status="all", limit=50):    
    headers ={"APCA-API-KEY-ID" : api_key, "APCA-API-SECRET-KEY":secret_key}
    params = {'status':status, 
              'limit':limit}
    
    r = requests.get(url = "https://paper-api.alpaca.markets/v2/orders", 
                      headers = headers, json = params)
    
    js = r.json()
    df = pd.DataFrame(js)
    return df

listaOrdenes = orders()
listaOrdenes.iloc[0]

## Consulta puntual  (GET)

In [ ]:
def getOrder(orderID):    
    headers ={"APCA-API-KEY-ID" : api_key, "APCA-API-SECRET-KEY":secret_key}
    r = requests.get(url = "https://paper-api.alpaca.markets/v2/orders/"+orderID, 
                      headers = headers)
    
    js = r.json()
    return js

getOrder('c3689e60-231a-4b63-9bfc-2385f98b0dfe')

## Modificación de orden  (PATCH)

In [ ]:
def replaceOrder(order_id, qty=None, time_in_force=None, limit_price=None):    
    headers ={"APCA-API-KEY-ID" : api_key, "APCA-API-SECRET-KEY":secret_key}
    params = {'qty':qty, 
              'time_in_force':time_in_force,
              'limit_price':limit_price}
    
    endpoint = "https://paper-api.alpaca.markets/v2/orders/"+order_id
    r = requests.patch(url = endpoint, headers = headers, json = params)
    
    js = r.json()
    return js

In [ ]:
replaceOrder(order_id='c3689e60-231a-4b63-9bfc-2385f98b0dfe', limit_price = 5.34)

La salida nos devuelve la orden remplazada

## Eliminar orden (DELETE)

Obviamente nos faltaba poder cancelar una orden por ID, esta es la mas sencilla de todas, solo hay que pasarle el ID y el método de request que usamos es DELETE

In [ ]:
def cancelOrder(order_id):    
    headers ={"APCA-API-KEY-ID" : api_key, "APCA-API-SECRET-KEY":secret_key}
    endpoint = "https://paper-api.alpaca.markets/v2/orders/"+order_id
    r = requests.delete(url = endpoint,headers = headers)

In [ ]:
cancelOrder('c3689e60-231a-4b63-9bfc-2385f98b0dfe')

## Ver Posiciones Abiertas de la cuenta

In [ ]:
def positions():    
    headers ={"APCA-API-KEY-ID" : api_key, "APCA-API-SECRET-KEY":secret_key}
    endpoint = "https://paper-api.alpaca.markets/v2/positions"
    r = requests.get(url = endpoint, headers = headers)
    js = r.json()
    return js
    
positions()

In [ ]:
df = pd.DataFrame(positions())

df = df.loc[:,['symbol','qty','side','avg_entry_price','market_value',
               'cost_basis','unrealized_pl','current_price','lastday_price']]
df

# Screener

## Listado de acitivos

In [ ]:
def assets():    
    headers ={"APCA-API-KEY-ID" : api_key, "APCA-API-SECRET-KEY":secret_key}
    r = requests.get(url = "https://paper-api.alpaca.markets/v2/assets", 
                      headers = headers)

    js = r.json()
    df = pd.DataFrame(js)
    df = df.loc[df.status=='active'].drop(['status','class','easy_to_borrow',
                                          'tradable','marginable'], axis=1)
    return df
activos = assets()

In [ ]:
activos

In [ ]:
activos.groupby('exchange').size()

In [ ]:
activos.groupby('shortable').size()

## WatchLists

## CRUD

### Crear listas

In [ ]:
def createWatchlist(name, symbols):    
    headers ={"APCA-API-KEY-ID" : api_key, "APCA-API-SECRET-KEY":secret_key}
    params = {'name':name, 'symbols':symbols}
    r = requests.post(url = "https://paper-api.alpaca.markets/v2/watchlists", 
                      headers = headers, json=params)
    
    js = r.json()
    df = pd.DataFrame(js['assets'])
    df.set_index('symbol', inplace=True)
    df = df.loc[df.status=='active'].drop(['status','class','easy_to_borrow',
                                          'tradable','marginable'], axis=1)
    return js['id'], df

bancos=['BBAR','BMA','SUPV']
listaID, lista = createWatchlist('bancos_arg', symbols=bancos)

In [ ]:
listaID

In [ ]:
lista

### Ver listas creadas

In [ ]:
def watchlist():    
    headers ={"APCA-API-KEY-ID" : api_key, "APCA-API-SECRET-KEY":secret_key}
    r = requests.get(url = "https://paper-api.alpaca.markets/v2/watchlists", 
                      headers = headers)

    js = r.json()
    df = pd.DataFrame(js)
    df.drop('account_id',axis=1, inplace=True)
    return df

watchlist()

In [ ]:
techs=['AAPL','NFLX','MSFT','FB','GOOGL']
createWatchlist('techs', symbols=techs)

In [ ]:
watchlist()

### Obtener watchList xID

In [ ]:
def getWatchlist(wID):    
    headers ={"APCA-API-KEY-ID" : api_key, "APCA-API-SECRET-KEY":secret_key}
    r = requests.get(url = "https://paper-api.alpaca.markets/v2/watchlists/"+wID, 
                      headers = headers)

    js = r.json()
    df = pd.DataFrame(js['assets'])
    df.set_index('symbol', inplace=True)
    df = df.loc[df.status=='active'].drop(['status','class','easy_to_borrow',
                                          'tradable','marginable'], axis=1)
    
    return df

getWatchlist('7b769561-efdc-4d77-8211-991ce6976b4d')

### Borrar watchList xID

In [ ]:
def deleteWatchlist(wID):    
    headers ={"APCA-API-KEY-ID" : api_key, "APCA-API-SECRET-KEY":secret_key}
    endpoint = "https://paper-api.alpaca.markets/v2/watchlists/"+wID
    r = requests.delete(url = endpoint, headers = headers)
    js = r.text
    return js

In [ ]:
watchlist()

In [ ]:
deleteWatchlist('7b769561-efdc-4d77-8211-991ce6976b4d')

In [ ]:
watchlist()

### Modificar watchList xID (PUT)

In [ ]:
def updateWatchlist(wID, lista):    
    headers ={"APCA-API-KEY-ID" : api_key, "APCA-API-SECRET-KEY":secret_key}
    params = {'symbols': lista}
    endpoint = "https://paper-api.alpaca.markets/v2/watchlists/"+wID
    r = requests.put(url = endpoint, headers = headers, json=params)
    
    return r

In [ ]:
getWatchlist('daf9e3b5-6373-412c-a076-5032b1904470')

In [ ]:
bancosNuevos = ['SUPV', 'GGAL']
updateWatchlist('daf9e3b5-6373-412c-a076-5032b1904470', bancosNuevos)

In [ ]:
getWatchlist('daf9e3b5-6373-412c-a076-5032b1904470')

In [ ]:
updateWatchlist('cf67f3d7-93a2-49be-b2ef-22766c38ab14', bancosNuevos)

### Agregar Ticker a una watchList existente (POST)

In [ ]:
def addTicker(wID, symbol):    
    headers ={"APCA-API-KEY-ID" : api_key, "APCA-API-SECRET-KEY":secret_key}
    params = {'symbol': symbol}
    endpoint = "https://paper-api.alpaca.markets/v2/watchlists/"+wID
    r = requests.post(url = endpoint, headers = headers, json=params)
    js = r.json()
    df = pd.DataFrame(js['assets'])
    df.set_index('symbol', inplace=True)
    df = df.loc[df.status=='active'].drop(['status','class','easy_to_borrow',
                                          'tradable','marginable'], axis=1)
    
    return df

In [ ]:
watchlist()

In [ ]:
getWatchlist('cf67f3d7-93a2-49be-b2ef-22766c38ab14')

In [ ]:
addTicker('cf67f3d7-93a2-49be-b2ef-22766c38ab14','AAPL')

# Ejercicios - Screener

Atención, la idea es terminar los 7 ejercicios porque están pensado como un bloque, es decir que todos forman parte de una especie de mini-screener para poder hacer una intro a este tema, aprovechando la funcionalidad de las watchlists de Alpaca podemos armar nuestro primer screener de 9000 activos de USA en 2 minutos via señal de trading predefinida y guardar los resultados como si se tratara de una alerta de trading

1- Partiendo de la función vista en clase para crear una watchList, mejorar la función para que (sin dejar de devolver el id y la lista en caso de estar todo ok), en caso de que se pase un ticker no listado devuelva al usuario el mensaje de error. Verificar su funcionamiento

2- Vamos a empezar a pensar en un potencial screener, para ello vamos a usar el endpoint "assets" para armar una función que me devuelva todos los tickers de la API y luego de ello con esos activos vamos a armar una gran lista con listas de a 200 activos, ya que la API me  permite ver series históricas trayendo de a 200 activos, para verificar el script vamos a imprimir la última lista y su cantidad de activos que debe ser menor de 200 y la cantidad de listas de 200
<br><br>A fines de poder usar este código mas adelante en un screener meter todo el código en una función: listasActivos()

3- Siguiendo en la línea de ir preparando el terreno para llegar a un screener, armar una función que si le paso una lista con listas de tickers me devuelva una lista con strings de tickers separados por comas, es decir:
 * Recibe: [ ['activo1','activo2','activo3'], ['activo4','activo5','activo_nn'] ]
 * Devuelve: [ 'activo1,activo2,activo3', 'activo4,activo5,activo_nn' ]
 
 Probar la función dando como entrada la salida de la función del ejercicio anterior

4- Agregarle lo necesario a prueba de fallos de la funcion getHistory definida en este capítulo para que acepte una lista de activos y me devuelve un diccionario cuyas claves sean las listas del ejercicio anterior y los valores unos dataFrames con el historial

Verificar el funcionamiento correcto de esta función pasándole como argumento la primera lista de 200 activos del ejercicio anterior en un timeframe de 1 minuto de los ultimos 60 dias

5- Seguimos armando el screener de a poco, paso a paso como quien no quiere la cosa los voy a ir llevando, confien en mi...

crear una función que tome de entrada un dataFrame de datos y me devuelva como salida un string que indique sobreCompra o SobreVenta en función de la siguiente lógica:
   * SobreCompra: Si  la media movil rapida (default 50) es + de un x% superior (default 3%) a la media lenta (default 200)
   * SobreVenta: Si  la media movil rapida (default 50) es + de un x% inferior (default 3%) a la media lenta (default 200)
   

6- Usando las funciones necesarias de los ejercicio anteriores crear un script que tome todos los activos de la API y me cree una watchList llamada "sobreCompra" y otra llamada "SobreVenta" con los activos que obviamente cumplan las condiciones enunciadas para tal configuración en el ejercicio previo

7- Usando las funciones vistas hasta acá armar un script que levante en una variable el ID de la watchList primaria (que es la que queda guardada en el FrontEnd de Alpaca del paper trading) y que agregue uno por uno los activos de la lista de sobrecompra definida en el ejercicio anterior simplemente para ver reflejado el resultado en la pantalla de https://app.alpaca.markets/paper/dashboard/overview el resultado del screener

Para ello los activos o lista de activos de sobrecompra deben ser levantados de la watchlist creada anteriormente llamada con ese nombre

8- Finalmente usando todas las funciones que vios hasta acá definir una función <b>screener_sobreCompra()</b> que simplemente tome como parámetros -fast -slow y -p que son los parámetros de ruedas de las medias móviles y el porcentaje por encima del cruce para disparar señal de sobrecompra, y que me actualice la watchList primaria con los activos que están en el filtro dado

Probar configurando en un 5% el screener con medias de 50 y 200 para testearlo y que den unas pocas acciones de las 9000 listadas e

# Respuestas

In [ ]:
#========================#
#      Ejercicio 1       #
#========================#


def createWatchlist(name, symbols):    
    headers ={"APCA-API-KEY-ID" : api_key, "APCA-API-SECRET-KEY":secret_key}
    params = {'name':name, 'symbols':symbols}
    r = requests.post(url = "https://paper-api.alpaca.markets/v2/watchlists", 
                      headers = headers, json=params)
    
    js = r.json()
    try:
        msg = js['id']
        df = pd.DataFrame(js['assets'])
        df.set_index('symbol', inplace=True)
        df = df.loc[df.status=='active']
        df =df.drop(['status','class','easy_to_borrow','tradable','marginable'],axis=1)
    except:
        df = pd.DataFrame()
        msg = js
    return msg, df

tickers = ['BBAR','BMA','TICKER_LOCO']
createWatchlist('lista_test', symbols = tickers)

In [ ]:
#========================#
#      Ejercicio 2       #
#========================#


def listasActivos():    
    headers ={"APCA-API-KEY-ID" : api_key, "APCA-API-SECRET-KEY":secret_key}
    endpoint = "https://paper-api.alpaca.markets/v2/assets"
    r = requests.get(url = endpoint, headers = headers)

    js = r.json()
    
    tickers = [row['symbol'] for row in js if row['status']=='active']
    listas = []
    for i in range(len(tickers)//200 + 1):
        listas.append(tickers[i*200:(i+1)*200])

    return listas

listas = listasActivos()

print('\n Ultima lista:\n',listas[len(listas)-1])
print('\n Cantidad de Ultima lista:',len(listas[len(listas)-1]))
print('\n Cantidad de listas de 200:',len(listas)-1)

In [ ]:
#========================#
#      Ejercicio 3       #
#========================#


def sublist_to_string(listas):
    for i in range(len(listas)):
        listas[i] = ','.join(listas[i])
    return listas

listas = sublist_to_string(listasActivos())
listas[0]

In [ ]:
#========================#
#      Ejercicio 4       #
#========================#


def getHistory(symbol, start, end, timeframe='1D'):    
    start = dt.datetime.strftime(start, format='%Y-%m-%dT%T-04:00')
    end = dt.datetime.strftime(end, format='%Y-%m-%dT%T-04:00')

    headers ={"APCA-API-KEY-ID" : api_key, "APCA-API-SECRET-KEY":secret_key}
    params =  {'symbols' : symbol, 'start':start, 'end':end}
    endpoint = "https://data.alpaca.markets/v1/bars/"+timeframe
    r = requests.get(url = endpoint, headers =headers, params=params)
    js = r.json()
    
    tickers = symbol.split(",")
    dfs = {}
    for ticker in tickers:
        # Bloque por los errores cuando no hay datos
        try:
            dfs[ticker] = pd.DataFrame(js[ticker])
            dfs[ticker].t = pd.to_datetime(dfs[ticker].t, unit='s')
            dfs[ticker].set_index(dfs[ticker].t, inplace=True)
        except:
            pass
    return dfs

import datetime as dt
end =  dt.date.today() + dt.timedelta(days=1)
start = end - dt.timedelta(days=6)

data = getHistory(listas[0], start, end, timeframe='1Min')

In [ ]:
len(data)

In [ ]:
data.keys()

In [ ]:
data['AAPL']

In [ ]:
#========================#
#      Ejercicio 5       #
#========================#


def signal(df, fast=50, slow=200, p=3):
    df['sma_f'] = df.c.rolling(fast).mean()
    df['sma_s'] = df.c.rolling(slow).mean()
    df['cruce'] = (df['sma_f']/df['sma_s']-1)*100
    
    alert = ''
    if ( float(df.iloc[-1:]['cruce']) < -p ):
        alert = 'SobreVenta'
    if ( float(df.iloc[-1:]['cruce']) > p ):
        alert = 'SobreCompra'
        
    return alert

In [ ]:
signal(data['AAPL'], fast=50, slow=200, p=3)

In [ ]:
#========================#
#      Ejercicio 6       #
#========================#

# Uso la funcion del ejercicio 3 y 2
# armo las listas de a 200 activos para traer la data lista por lista
listas = sublist_to_string(listasActivos())


import datetime as dt
end =  dt.date.today()  + dt.timedelta(days=1)
start = end - dt.timedelta(days=6)

sc = []
sv = []

for i in range(len(listas)):
    
    # Para ver la evolución ya que tarda unos 2 minutos en correr
    print(i,'..',end='')
    
    # Uso la función del ejercicio 4
    data = getHistory(listas[i], start, end, timeframe='1Min')
    for ticker in data:
        try:
            # Uso la función del ejercicio 5
            s = signal(data[ticker], fast=10, slow=200, p=6)
            
            if s=='SobreCompra':
                sc.append(ticker)
            if s=='SobreVenta':
                sv.append(ticker)
        except:
            pass

# Uso la función del ejercicio 1
wSC = createWatchlist('sobreCompra', sc)
wSV = createWatchlist('sobreVenta', sv)

print('\n\nSobreCompra ID:',wSC[0],'SobreVenta ID:',wSV[0])
print('\n\nSobreCompra Cantidad:',len(wSC[1]),'SobreVenta Cantidad:',len(wSV[1]))

In [ ]:
watchlist()

In [ ]:
#========================#
#      Ejercicio 7       #
#========================#


def updateWatchlist(wID, lista):    
    headers ={"APCA-API-KEY-ID" : api_key, "APCA-API-SECRET-KEY":secret_key}
    params = {'symbols': lista}
    r = requests.put(url = "https://paper-api.alpaca.markets/v2/watchlists/"+wID, 
                      headers = headers, json=params)
    
    return r


# Armo lista c/las que estan guardadas en el Watchlist SobreCompra
listas = watchlist().set_index('name')
sobreCompraID = listas.loc['sobreCompra','id']
sobreCompraDF = getWatchlist(sobreCompraID)
sc = list(sobreCompraDF.index)

# Busco el ID de la primaria
listas = watchlist().set_index('name')
primariaID = listas.loc['Primary Watchlist','id']

# Lleno la primaria con las de sobreCompra
upd = updateWatchlist(primariaID, lista=sc)


In [ ]:
updateWatchlist(primariaID, [])

In [ ]:
#========================#
#      Ejercicio 8       #
#========================#

import datetime as dt, pandas as pd, requests

def screener_sobreCompra(fast, slow, p):
    listas = sublist_to_string(listasActivos())
    end =  dt.date.today()  + dt.timedelta(days=1)
    start = end - dt.timedelta(days=6)

    sc = []
    for i in range(len(listas)):

        print(i,'..',end='')

        data = getHistory(listas[i], start, end, timeframe='1Min')
        for ticker in data:
            try:
                s = signal(data[ticker], fast=fast, slow=slow, p=p)
                if s=='SobreCompra':
                    sc.append(ticker)
            except:
                pass

    listas = watchlist().set_index('name')
    primariaID = listas.loc['Primary Watchlist','id']

    upd = updateWatchlist(primariaID, lista=sc)
    
    
screener_sobreCompra(fast=10, slow=200, p=7)

# Bases de Datos introducción

* claves foraneas
* claves primarias

<img src='imagenes/BBDD.jpeg' width='600'>

Y así sucesivamente se pueden ir creando nuevas entidades que se van relacionando con otras

En este caso vamos a agregar a nuestro modelo de subyacentes y derivados, una entidad de "fondos" que podrian ser FCIs y "clientes" que podrian ser las cuentas de un Alyc

<img src='imagenes/BBDD2.jpeg' width='600'>